## RDP Middle Office News Use Case
### Exploring RDP News Filters
How a user can make API Req/Res call for news:
* Published in the last 24 hours, where
* The messages are tagged with PermIds (1, 2, 3, 4)
* And the messages contain topic codes (x, y or z) 

Get started by absorbing valid RDP credentials

In [1]:
%run ../creds/credentials.ipynb

###  Setup for RDP API Access

In [2]:
import requests, json, time, getopt, sys, pandas as pd
import urllib.parse

# Application Constants
RDP_version = "/v1"
base_URL = "https://api.refinitiv.com"
category_URL = "/auth/oauth2"
endpoint_URL = "/token"
CLIENT_SECRET = ""
SCOPE = "trapi"
TOKEN_FILE = "token.txt"
TOKEN_ENDPOINT = base_URL + category_URL + RDP_version + endpoint_URL

### Setup for Max Display - a Convenience

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

### Obtain a Valid RDP Authentication Token 

In [5]:
def _requestNewToken(refreshToken):
    if refreshToken is None:
        tData = {
            "username": RDP_LOGIN,
            "password": RDP_PASSWORD,
            "grant_type": "password",
            "scope": SCOPE,
            "takeExclusiveSignOnControl": "true"
        };
    else:
        tData = {
            "refresh_token": refreshToken,
            "grant_type": "refresh_token",
        };

    # Make a REST call to get latest access token
    response = requests.post(
        TOKEN_ENDPOINT,
        headers = {
            "Accept": "application/json"
        },
        data = tData,
        auth = (
            APP_KEY,
            CLIENT_SECRET
        )
    )
    
    if response.status_code != 200:
        raise Exception("Failed to get access token {0} - {1}".format(response.status_code, response.text));

    # Return the new token
    return json.loads(response.text);

def saveToken(tknObject):
    tf = open(TOKEN_FILE, "w+");
    print("Saving the new token");
    # Append the expiry time to token
    tknObject["expiry_tm"] = time.time() + int(tknObject["expires_in"]) - 10;
    # Store it in the file
    json.dump(tknObject, tf, indent=4)

def getToken():
    try:
        print("Reading the token from: " + TOKEN_FILE);
        # Read the token from a file
        tf = open(TOKEN_FILE, "r+")
        tknObject = json.load(tf);

        # Is access token valid
        if tknObject["expiry_tm"] > time.time():
            # return access token
            return tknObject["access_token"];

        print("Token expired, refreshing a new one...");
        tf.close();
        # Get a new token from refresh token
        tknObject = _requestNewToken(tknObject["refresh_token"]);

    except Exception as exp:
        print("Caught exception: " + str(exp))
        print("Getting a new token using Password Grant...");
        tknObject = _requestNewToken(None);

    # Persist this token for future queries
    saveToken(tknObject)
    print("Token is: " + tknObject["access_token"])
    # Return access token
    return tknObject["access_token"];    
    
token = getToken()
token

Reading the token from: token.txt
Token expired, refreshing a new one...
Caught exception: Failed to get access token 400 - {"error":"invalid_grant"   } 
Getting a new token using Password Grant...
Saving the new token
Token is: eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImRMdFd2Q0tCSC1NclVyWm9YMXFod2pZQ2t1eDV0V2ZSS2o4ME9vcjdUY28ifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwiME1mX2RJWm1SVW5OYTc3dTlWR25XTXdLRGRvaWJqbnJpM0FOT1NweVdNcVZfb2wzMlRnOVZ3OFQtazl1YmdDMkNybTBIS2szRVVVVjZUaFRua0tBTHVCVndfSERZUE5OT1FzRWZ4bGEyWm1tUHdBMmliMm1pSU9vN1RxRXJmTVFoWkczMmFNYlhvRTQ0cVgzeG1HQm5DWUJVQmljVklUajE4UDZXVVdiOHA2aGNOOFhyUVRYa1M3WXRvMUgteE5VMEpWSVBKZE5ZcW41QS1vR1VORmV5NzlMWDlNUEFUWEU4dXVpWHlNV295MFoxT1BsRWRsMVFJal9oVmlmOUdXWU1WUEdsRlU2NXBuRmluVWh3Wnd2ZUIxaFBXUlJiZHJmNHpaVUlvX2xGMVFtd0hob0xkTHd2Ui0wYV9iZzFHeTdEaExycGd5N0NMZk01NEhqQ1lEZF82dmtvOWFsTEdmWnNYdGx5Y1RPX1F1OG9MM1BkbUtQQVZmQTh3LXd4Rkl2aTM4OVlHVWNycl9CblJWOXRTZnVsUThhUDkydnVRY2I2T3BEb201ODZ4NW5FUktqM002ZTFuOFFVdm1pWVZoRUFJcG81TzlUYUxTLWlpWnpXWjNz

'eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImRMdFd2Q0tCSC1NclVyWm9YMXFod2pZQ2t1eDV0V2ZSS2o4ME9vcjdUY28ifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwiME1mX2RJWm1SVW5OYTc3dTlWR25XTXdLRGRvaWJqbnJpM0FOT1NweVdNcVZfb2wzMlRnOVZ3OFQtazl1YmdDMkNybTBIS2szRVVVVjZUaFRua0tBTHVCVndfSERZUE5OT1FzRWZ4bGEyWm1tUHdBMmliMm1pSU9vN1RxRXJmTVFoWkczMmFNYlhvRTQ0cVgzeG1HQm5DWUJVQmljVklUajE4UDZXVVdiOHA2aGNOOFhyUVRYa1M3WXRvMUgteE5VMEpWSVBKZE5ZcW41QS1vR1VORmV5NzlMWDlNUEFUWEU4dXVpWHlNV295MFoxT1BsRWRsMVFJal9oVmlmOUdXWU1WUEdsRlU2NXBuRmluVWh3Wnd2ZUIxaFBXUlJiZHJmNHpaVUlvX2xGMVFtd0hob0xkTHd2Ui0wYV9iZzFHeTdEaExycGd5N0NMZk01NEhqQ1lEZF82dmtvOWFsTEdmWnNYdGx5Y1RPX1F1OG9MM1BkbUtQQVZmQTh3LXd4Rkl2aTM4OVlHVWNycl9CblJWOXRTZnVsUThhUDkydnVRY2I2T3BEb201ODZ4NW5FUktqM002ZTFuOFFVdm1pWVZoRUFJcG81TzlUYUxTLWlpWnpXWjNzdkgtV2tzbko0SDBNNTJOVzJiTDNGRWhQTjZmanZuS3RiRl9EbGstbzQyaFZ0cWdQbXk2NlowdzA5UVhVWmp2VlNUSFRhUWhSQWlkMndUN21tdEZBM3hZSWhzVzR5RXczaFR6emdOQkVMeWVrVFV0ZnNtRklaQ1E3QV81anpsMk9nNERZY3h0dHJwUzkwQnFsRW5Yc0hZbkxmemVteXhyUmR2S0EwWk5CTUZT

### Request News Headlines - Define a Function 

In [6]:
def getNewsHeadlines(query, numLines, return_as_text=False):
    news_category_URL = "/data/news"
    headlines_endpoint_URL = "/headlines?"

    REQUEST_URL = base_URL + news_category_URL + RDP_version + headlines_endpoint_URL+ query # +"&limit="+ str(numLines);
    
    accessToken = getToken();
    print("Requesting: ",REQUEST_URL)
    
    acceptValue = "*/*"
    dResp = requests.get(REQUEST_URL, headers = {"Authorization": "Bearer " + accessToken, "Accept": acceptValue});
    
    if dResp.status_code != 200:
        print("Unable to get data. Code %s, Message: %s" % (dResp.status_code, dResp.text));
        if dResp.status_code != 401:   # error other then token expired
            return("Error "+str(dResp.status_code)) 
        accessToken = getToken();     # token refresh on token expired
        dResp = requests.get(REQUEST_URL, headers = {"Authorization": "Bearer " + accessToken, "Accept": acceptValue});
        if dResp.status_code == 200:
            print("Resource access successful")
    else:
        print("Resource access successful")
    if return_as_text:
        return dResp.text
    jResp = json.loads(dResp.text);
    return jResp

### Request Headlines per Filter Requirements (by PermIds and Topics, for the last 24 hrs)

In [13]:
N = 100
query = 'query=(4295905573, 5030853586) and (searchIn:HeadlineOnly) and (last 24 hours)';
#query = 'query=(4295905573, 5030853586) and (Topic:E:F) and (searchIn:HeadlineOnly) and (last 24 hours)';
#query = 'query=(4295905573, 5030853586) and (Topic:SIG OR Topic:WEA) and (searchIn:FullStory) and (last 24 hours)';
#query = 'query=(4295905573, 5030853586) and (Topic:E:F) and (searchIn:HeadlineOnly) and (last 24 hours)';
jRespHeadlines = getNewsHeadlines(query, N)

dataAll = []
dfAll = pd.DataFrame()
iterations_count = 0;
while True:
    cursor = ""
    dfH = pd.json_normalize(jRespHeadlines,record_path =['data'])
    if not dfH.empty:
        dataAll.append(dfH)
        if 'meta' in jRespHeadlines:
            if 'next' in jRespHeadlines['meta']:
                # print('Meta.next=', jRespHeadlines['meta']['next'])
                cursor = urllib.parse.quote(jRespHeadlines['meta']['next'])
                print('Iteration= ',iterations_count,'Cursor= ', cursor)   
                query = 'cursor='+cursor;
    else:
        print('No results that qualify filter requirements')
    if not cursor:
        break
    iterations_count += 1
    jRespHeadlines = getNewsHeadlines(query, N)

if dataAll:    
    dfAll = pd.concat(dataAll,ignore_index=True)
    print('Displaying full result:')
    display(dfAll)

Reading the token from: token.txt
Requesting:  https://api.refinitiv.com/data/news/v1/headlines?query=(4295905573, 5030853586) and (searchIn:HeadlineOnly) and (last 24 hours)
Resource access successful
Iteration=  0 Cursor=  H4sIAAAAAAAAAI2PQWuEMBCF/8tc15ZEzarpSXqwC2VdxFvpIZhskWYTidFuEf97Jwi7lF46h5DMfPPy3gLnXnvlgC9gB%2BWEt3gHPKLtbeQI/G2BWehJ4aThZXl6faxx7r%2BH0DHqa%2BysVLBGd%2BzE07hgBWEsS%2B4oCl56%2BQtseFXX1X8EGUlIzhKW7/8Kvt86txBrBBdx7exkPHBKIpDChzwfKmQdvR2Qjkkck5QULU1IltIdwQqbQz9bH7jgBT/gQPdpwXKWYaW5OTblZ0apNbtNgZIn8vD8cmibA5rrx6O64jp4N4UUMFrnt2CtrbVExCmtZmE6VTk7BSdCa1h/ACzG5EOOAQAA
Reading the token from: token.txt
Requesting:  https://api.refinitiv.com/data/news/v1/headlines?cursor=H4sIAAAAAAAAAI2PQWuEMBCF/8tc15ZEzarpSXqwC2VdxFvpIZhskWYTidFuEf97Jwi7lF46h5DMfPPy3gLnXnvlgC9gB%2BWEt3gHPKLtbeQI/G2BWehJ4aThZXl6faxx7r%2BH0DHqa%2BysVLBGd%2BzE07hgBWEsS%2B4oCl56%2BQtseFXX1X8EGUlIzhKW7/8Kvt86txBrBBdx7exkPHBKIpDChzwfKmQdvR2Qjkkck5QULU1IltIdwQqbQz9bH7jgBT/gQPdpwXKWYaW5OTblZ0apNbtNgZIn8vD8cmibA5r

,storyId,newsItem._version,newsItem.contentMeta.audience,newsItem.contentMeta.creator,newsItem.contentMeta.infoSource,newsItem.contentMeta.language,newsItem.contentMeta.subject,newsItem.contentMeta.urgency.$,newsItem.itemMeta.firstCreated.$,newsItem.itemMeta.versionCreated.$,newsItem.itemMeta.title
0,urn:newsml:reuters.com:20220410:nNRAk73gr2:1,1,"[{'_qcode': 'NP:NWR1'}, {'_qcode': 'NP:NWR2'}, {'_qcode': 'NP:NWRM'}]","[{'_qcode': 'NS:SUEDDE', '_role': 'sRole:source'}]","[{'_qcode': 'NS:SUEDDE', '_role': 'sRole:source'}, {'_qcode': 'NS:SUEDDE', '_role': 'sRole:origProv'}]",[{'_tag': 'de'}],"[{'_qcode': 'R:EA.O'}, {'_qcode': 'B:255'}, {'_qcode': 'G:4'}, {'_qcode': 'B:87'}, {'_qcode': 'M:2CY'}, {'_qcode': 'B:259'}, {'_qcode': 'B:69'}, {'_qcode': 'R:BAC.N'}, {'_qcode': 'B:291'}, {'_qcode': 'G:9'}, {'_qcode': 'B:278'}, {'_qcode': 'B:1804'}, {'_qcode': 'M:1WJ'}, {'_qcode': 'B:173'}, {'_qcode': 'M:1WN'}, {'_qcode': 'M:2DZ'}, {'_qcode': 'B:161'}, {'_qcode': 'B:92'}, {'_qcode': 'B:290'}, {'_qcode': 'G:6J'}, {'_qcode': 'R:DIS.N'}, {'_qcode': 'B:172'}, {'_qcode': 'M:2DY'}, {'_qcode': 'B:95'}, {'_qcode': 'M:Z'}, {'_qcode': 'B:1446'}, {'_qcode': 'M:1QD'}, {'_qcode': 'M:2CR'}, {'_qcode': 'R:GOOGL.O'}, {'_qcode': 'M:1WL'}, {'_qcode': 'B:94'}, {'_qcode': 'B:86'}, {'_qcode': 'M:1P2'}, {'_qcode': 'R:NFLX.O'}, {'_qcode': 'R:AAPL.O'}, {'_qcode': 'P:5064610769'}, {'_qcode': 'P:5030853586'}, {'_qcode': 'P:4295901526'}, {'_qcode': 'P:4295902158'}, {'_qcode': 'P:4295915611'}, {'_qcode': 'P:8589934339'}, {'_qcode': 'P:4295905573'}]",3,2022-04-10T12:15:08.073Z,2022-04-10T12:15:08.073Z,"[{'$': 'Unterhaltung: Spielen mit Netflix, wetten mit Disney'}]"
1,urn:newsml:reuters.com:20220410:nNRAk73csc:1,1,"[{'_qcode': 'NP:NWR1'}, {'_qcode': 'NP:NWR2'}, {'_qcode': 'NP:NWRM'}, {'_qcode': 'NP:W2VEC'}]","[{'_qcode': 'NS:SOUTHC', '_role': 'sRole:source'}]","[{'_qcode': 'NS:SOUTHC', '_role': 'sRole:source'}, {'_qcode': 'NS:SOUTHC', '_role': 'sRole:origProv'}]",[{'_tag': 'en'}],"[{'_qcode': 'M:2CU'}, {'_qcode': 'G:4'}, {'_qcode': 'G:AJ'}, {'_qcode': 'G:E'}, {'_qcode': 'G:A'}, {'_qcode': 'E:C'}, {'_qcode': 'M:N5'}, {'_qcode': 'M:2CM'}, {'_qcode': 'G:9'}, {'_qcode': 'M:E7'}, {'_qcode': 'M:2CQ'}, {'_qcode': 'M:N'}, {'_qcode': 'M:2CT'}, {'_qcode': 'M:2CX'}, {'_qcode': 'M:28W'}, {'_qcode': 'M:I4'}, {'_qcode': 'M:J2'}, {'_qcode': 'G:3H'}, {'_qcode': 'M:2CP'}, {'_qcode': 'M:2CS'}, {'_qcode': 'G:6'}, {'_qcode': 'G:B1'}, {'_qcode': 'G:S'}, {'_qcode': 'G:38'}, {'_qcode': 'G:K'}, {'_qcode': 'G:8W'}, {'_qcode': 'M:21P'}, {'_qcode': 'M:1QD'}, {'_qcode': 'M:2CR'}, {'_qcode': 'G:1'}, {'_qcode': 'M:1Y7'}, {'_qcode': 'G:71'}, {'_qcode': 'G:B'}, {'_qcode': 'P:5030853586'}, {'_qcode': 'P:4295899948'}, {'_qcode': 'P:5001388183'}, {'_qcode': 'P:5035931441'}]",3,2022-04-10T12:11:14.018Z,2022-04-10T12:11:14.018Z,[{'$': 'Hong Kong's public sector should lead the effort to narrow the gender pay gap'}]
2,urn:newsml:reuters.com:20220410:nNRAk73csf:1,1,"[{'_qcode': 'NP:NWR1'}, {'_qcode': 'NP:NWR2'}, {'_qcode': 'NP:NWRM'}, {'_qcode': 'NP:W2VEC'}]","[{'_qcode': 'NS:SOUTHC', '_role': 'sRole:source'}]","[{'_qcode': 'NS:SOUTHC', '_role': 'sRole:source'}, {'_qcode': 'NS:SOUTHC', '_role': 'sRole:origProv'}]",[{'_tag': 'en'}],"[{'_qcode': 'M:2CS'}, {'_qcode': 'M:1YI'}, {'_qcode': 'M:2CU'}, {'_qcode': 'G:4'}, {'_qcode': 'G:G'}, {'_qcode': 'G:A'}, {'_qcode': 'M:2CM'}, {'_qcode': 'M:E7'}, {'_qcode': 'M:2CQ'}, {'_qcode': 'M:1QD'}, {'_qcode': 'M:2CR'}, {'_qcode': 'M:N'}, {'_qcode': 'G:3'}, {'_qcode': 'M:2CT'}, {'_qcode': 'M:2CX'}, {'_qcode': 'M:B'}, {'_qcode': 'G:CG'}, {'_qcode': 'G:3L'}, {'_qcode': 'G:7J'}, {'_qcode': 'M:2CP'}, {'_qcode': 'P:5001388183'}, {'_qcode': 'P:34425475154'}, {'_qcode': 'P:34413819559'}, {'_qcode': 'P:5030853586'}, {'_qcode': 'P:4295899948'}]",3,2022-04-10T12:11:09.939Z,2022-04-10T12:11:09.939Z,[{'$': 'Hongkongers who served in Falklands War will not be forgotten'}]
3,urn:newsml:reuters.com:20220410:nRscf7R8Wa:1,1,[{'_qcode': 'NP:RSECH'}],"[{'_qcode': 'NS:RSCH', '_role': 'sRole:sou

### Request a Story - Define a Function  

In [14]:
def getStory(storyId, jsonOrHtml):
    news_category_URL = "/data/news"
    story_endpoint_URL = "/stories"

    REQUEST_URL = base_URL + news_category_URL + RDP_version + story_endpoint_URL+ "/" + storyId 

    accessToken = getToken();
    print("Requesting: ",REQUEST_URL)
    
    acceptValue = "application/json"
    if jsonOrHtml != True:
        acceptValue = "text/html"
    dResp = requests.get(REQUEST_URL, headers = {"Authorization": "Bearer " + accessToken, "Accept": acceptValue});
    if dResp.status_code != 200:
        print("Unable to get data. Code %s, Message: %s" % (dResp.status_code, dResp.text));
        if dResp.status_code != 401:   # error other then token expired
            return("") 
        accessToken = getToken();     # token refresh on token expired
    else:
        print("Resource access successful")
        return dResp.text

### Request Story in Json (1st)

In [15]:
txt = getStory(dfAll.iloc[[0]]['storyId'].item(),True)
#txt = getStory('urn:newsml:reuters.com:20210811:nL1N2PI1YN:9')
#txt = getStory('urn:newsml:reuters.com:20210811:nTNF29sf:1', True)
jResp = json.loads(txt);
print(json.dumps(jResp, indent=2));

Reading the token from: token.txt
Requesting:  https://api.refinitiv.com/data/news/v1/stories/urn:newsml:reuters.com:20220410:nNRAk73gr2:1
Resource access successful
{
  "newsItem": {
    "_conformance": "power",
    "_guid": "tag:reuters.com,2022-04-10:newsml_NRAk73gr2",
    "_standard": "NewsML-G2",
    "_standardversion": "2.18",
    "_version": 1,
    "catalogRef": [
      {
        "_href": "http://xml.media.reuters.com/g2-standards/catalogs/ReutersMedia_G2-Standards-Catalog_v1.xml"
      }
    ],
    "rightsInfo": [
      {
        "copyrightHolder": {
          "_literal": ""
        },
        "copyrightNotice": [
          {
            "$": "Copyright \u00a9 2022 DIZdigital: Alle Rechte vorbehalten - S\u00fcddeutsche Zeitung Digitale Medien GmbH, Jegliche Ver\u00f6ffentlichung exklusiv \u00fcber www.sz-content.de"
          }
        ]
      }
    ],
    "itemMeta": {
      "itemClass": {
        "_qcode": "ninat:text",
        "_rtr:msgType": "S"
      },
      "provider": {

### Request Story for Display ( 1st)

In [16]:
txt = getStory(dfAll.iloc[[0]]['storyId'].item(), False)
print('********* story text')
print(txt)  
print('********* story HTML')
from IPython.core.display import display, HTML
display(HTML(txt))

Reading the token from: token.txt
Requesting:  https://api.refinitiv.com/data/news/v1/stories/urn:newsml:reuters.com:20220410:nNRAk73gr2:1
Resource access successful
********* story text
{"newsItem":{"_conformance":"power","_guid":"tag:reuters.com,2022-04-10:newsml_NRAk73gr2","_standard":"NewsML-G2","_standardversion":"2.18","_version":1,"catalogRef":[{"_href":"http://xml.media.reuters.com/g2-standards/catalogs/ReutersMedia_G2-Standards-Catalog_v1.xml"}],"rightsInfo":[{"copyrightHolder":{"_literal":""},"copyrightNotice":[{"$":"Copyright © 2022 DIZdigital: Alle Rechte vorbehalten - Süddeutsche Zeitung Digitale Medien GmbH, Jegliche Veröffentlichung exklusiv über www.sz-content.de"}]}],"itemMeta":{"itemClass":{"_qcode":"ninat:text","_rtr:msgType":"S"},"provider":{"_literal":"reuters.com"},"versionCreated":{"$":"2022-04-10T12:15:08.073Z"},"firstCreated":{"$":"2022-04-10T12:15:08.073Z"},"pubStatus":{"_qcode":"stat:usable"},"role":{"_qcode":"itemRole:N"},"title":[{"$":"Unterhaltung: Spielen

C:\Users\U8005173\AppData\Local\Temp\ipykernel_45984\687116073.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
